# Load and read data

In [1]:
import functools
import json
import os
import re
from typing import Mapping, Union
import yaml

import click
import numpy as np
from slicedimage import ImageFormat

import starfish.core.util.try_import
from starfish.experiment.builder import FetchedTile, TileFetcher, write_experiment_json
from starfish.types import Axes, Coordinates, CoordinateValue, Features

from czifile import imread, CziFile
from glob import glob

from skimage.transform import resize

Rename data to: Hybridization_*round* _ *gene* _ *fov*.npy 

In [ ]:
path = "Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/"
fov = 0
for file in os.listdir(path):
    if file.startswith('20'):
        czifile = CziFile(path+file)
        image = np.squeeze(czifile.asarray())
        metadata = czifile.metadata()
        round_ = file[13:14]
        genes = ['3GAPDH','5GAPDH', 'nuclei']
        for ch in range(image.shape[0]):
            #Hybridization1_Aldoc_fov_53.npy
            file_name = 'Hybridization'+str(round_)+'_'+genes[ch]+'_fov_'+str(fov)+'.npy'
            store = resize(image[ch],(3622,3627))
            np.save(path+'prepared/'+file_name, np.expand_dims(store,axis =0))
        #fov+=1

Check data shape

In [ ]:
path_ = 'Z://Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/output/'
for image in os.listdir(path_):
    if image.endswith('.tiff'):
        he = imread(path_ + image)
        print(np.squeeze(he).shape, image)
        #imsave(path_+image,np.squeeze(he) )

Create metadata file

In [2]:
meta = {'ImageProperties':{'HybImageSize': {'zcount' : 1},'PixelSize': {3632 , 3622, 1}}, 
           'HybridizationsInfos': {'Hybridization1': {'3GAPDH':'Alexa Fluor 594', '5GAPDH':'Alexa Fluor 546', 'nuclei':'dapi' }, 'Hybridization2':{'3GAPDH':'Alexa Fluor 594', '5GAPDH':'Alexa Fluor 546', 'nuclei':'dapi'}, 'Hybridization3' :{'3GAPDH':'Alexa Fluor 594', '5GAPDH':'Alexa Fluor 546', 'nuclei':'dapi'} }, 
           'TilesPositions':{'Hybridization1':{0:{'x':0,'y':0,'z':233.006}}, 'Hybridization2':{0:{'x':0,'y':0,'z':-2255.129}}, 'Hybridization3':{0:{'x':0,'y':0,'z':725.12}} }
}

In [3]:
with open('Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/prepared/Experimental_metadata.yml', 'w') as outfile:
    yaml.dump(meta, outfile, default_flow_style=True)

# Create osmFISHTile class

In [4]:
# We use this to cache images across tiles.  In the case of the osmFISH data set, volumes are saved
# together in a single file.  To avoid reopening and decoding the NPY file, we use a single-element
# cache that maps between file_path and the npy file.
@functools.lru_cache(maxsize=1)
def cached_read_fn(file_path) -> np.ndarray:
    return np.load(file_path)


class osmFISHTile(FetchedTile):

    def __init__(
            self,
            file_path: str,
            coordinates: Mapping[Union[str, Coordinates], CoordinateValue],
            z: int
    ) -> None:
        """Parser for an osmFISH tile.

        Parameters
        ----------
        file_path : str
            location of the osmFISH tile
        coordinates : Mapping[Union[str, Coordinates], CoordinateValue]
            the coordinates for the selected osmFISH tile, extracted from the metadata
        z : int
            the z-layer for the selected osmFISH tile
        """
        self.file_path = file_path
        self.z = z
        self._coordinates = coordinates

    @property
    def shape(self) -> Mapping[Axes, int]:
        """
        Gets image shape directly from the data. Note that this will result in the data being
        read twice, since the shape is retrieved from all tiles before the data is read, and thus
        single-file caching does not resolve the duplicated reads.

        Because the data here isn't tremendously large, this is acceptable in this instance.
        """
        raw_shape = self.tile_data().shape
        return {Axes.Y: raw_shape[0], Axes.X: raw_shape[1]}

    @property
    def coordinates(self) -> Mapping[Union[str, Coordinates], CoordinateValue]:
        return self._coordinates

    def tile_data(self) -> np.ndarray:
        return cached_read_fn(self.file_path)[self.z]  # slice out the correct z-plane

# Create osmFISHTileFetcher class

In [5]:
class osmFISHTileFetcher(TileFetcher):

    @starfish.core.util.try_import.try_import({"yaml"})
    def __init__(self, input_dir: str, metadata_yaml) -> None:
        """Implement a TileFetcher for an osmFISH experiment.

        This TileFetcher constructs spaceTx format for one or more fields of view, where
        `input_dir` is a directory containing all .npy image files and whose file names have the
        following structure:

        Hybridization<round>_<target>_fov_<fov_number>.npy

        Notes
        -----
        - osmFISH is a linearly multiplexed method. As such, each target is specified by a
          (channel, round) tuple. The files do not contain channel information.
        - This TileFetcher is specifically tailored to the gene panel used for a specific
          experiment. Generalization of this TileFetcher will require reimplementation of the
          `channel_map` method.
        """
        import yaml

        with open(metadata_yaml, "r") as f:
            self.osmfish_metadata = yaml.load(f)
        self.num_z = self.osmfish_metadata['ImageProperties']['HybImageSize']['zcount']
        self.input_dir = input_dir

    @property
    def channel_map(self) -> Mapping[str, int]:
        return {
            "Alexa Fluor 594": 0,
            "Alexa Fluor 546": 1,
            "dapi": 2,
        }

    @property
    def target_map(self) -> Mapping:
        """
        Parse the metadata to map channel number and round to the correct target, which identifies
        the right tile
        """
        parsed_metadata = {}
        for round_, round_data in self.osmfish_metadata['HybridizationsInfos'].items():

            # the metadata references a larger corpus of data than we use for this example so as we
            # iterate over the metadata, some rounds will not be found. In those cases, we just
            # continue through the loop without adding to parsed_metadata
            round_match = re.match(r"Hybridization(\d{1,2})", round_)
            
            if round_match is None:
                continue

            round_id = int(round_match.group(1)) - 1
            for target_name, fluorophore in round_data.items():
                if fluorophore in {"dapi"}:
                    continue
                channel = self.channel_map[fluorophore]
                parsed_metadata[round_id, channel] = target_name

        return parsed_metadata

    @property
    def fov_map(self) -> Mapping[int, str]:
        """This example is pared down to only 3 fovs, which are mapped to sequential integers"""
        return {
            0: "0",
        }

    def coordinate_map(self, round_: int, z: int):
        #pixel_size = self.osmfish_metadata['ImageProperties']['PixelSize']
        x_size, z_size, y_size = self.osmfish_metadata['ImageProperties']['PixelSize']
        #y_pixels, x_pixels, z_pixels = (
            #self.osmfish_metadata['ImageProperties']['HybImageSize'].values()
         #   self.osmfish_metadata['TilesPositions']['Hybridization1'][0].values()
        #)
        #y_size = y_pixels * pixel_size
        #x_size = x_pixels * pixel_size
        #z_size = z_pixels * pixel_size
        
        #position_string = (
         #   self.osmfish_metadata['TilesPositions'][f'Hybridization{round_ + 1}'][z]
        #)
        #y_pos, x_pos, z_pos = (float(v) for v in position_string.split(", "))
        #x_pos, y_pos, z_pos = self.osmfish_metadata['TilesPositions'][f'Hybridization{round_ + 1}'][z]
        x_pos, y_pos, z_pos = self.osmfish_metadata['TilesPositions']['Hybridization1'][0].values()
        return {
            Coordinates.X: (x_pos, x_pos + x_size),
            Coordinates.Y: (y_pos, y_pos + y_size),
            Coordinates.Z: (z_pos, z_pos + z_size),
        }

    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        
        target = self.target_map[round_label, ch_label]
        fov = self.fov_map[fov_id]
        basename =  f"Hybridization{round_label + 1}_{target}_fov_{fov}.npy"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = self.coordinate_map(round_label, zplane_label)
        return osmFISHTile(file_path, coordinates, zplane_label)

    def generate_codebook(self):
        mappings = []
        for (round_, channel), target in self.target_map.items():
            mappings.append({
                Features.CODEWORD: [{
                    Axes.ROUND.value: round_, Axes.CH.value: channel, Features.CODE_VALUE: 1
                }],
                Features.TARGET: target
            })
        return {
            "version": "0.0.0",
            "mappings": mappings
        }

In [6]:
class NucleiTileFetcher(TileFetcher):

    def __init__(self, input_dir: str, metadata_yaml) -> None:
        with open(metadata_yaml, "r") as f:
            self.osmfish_metadata = yaml.load(f)
        self.num_z = self.osmfish_metadata['ImageProperties']['HybImageSize']['zcount']
        self.input_dir = input_dir
        
    @property
    def fov_map(self) -> Mapping[int, str]:
        """This example is pared down to only 3 fovs, which are mapped to sequential integers"""
        return {
            0: "0",
        }
    def coordinate_map(self, round_: int, z: int):

        x_size, z_size, y_size = self.osmfish_metadata['ImageProperties']['PixelSize']
        x_pos, y_pos, z_pos = self.osmfish_metadata['TilesPositions']['Hybridization1'][0].values()
        return {
            Coordinates.X: (x_pos, x_pos + x_size),
            Coordinates.Y: (y_pos, y_pos + y_size),
            Coordinates.Z: (z_pos, z_pos + z_size),
        }

    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        fov = self.fov_map[fov_id]
        target = 'nuclei'
        basename =  f"Hybridization{round_label + 1}_{target}_fov_{fov}.npy"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = self.coordinate_map(round_label, zplane_label)
        return osmFISHTile(file_path, coordinates, zplane_label)

In [5]:
class DotsTileFetcher(TileFetcher):

    def __init__(self, input_dir: str, metadata_yaml) -> None:
        with open(metadata_yaml, "r") as f:
            self.osmfish_metadata = yaml.load(f)
        self.num_z = self.osmfish_metadata['ImageProperties']['HybImageSize']['zcount']
        self.input_dir = input_dir
        
    @property
    def fov_map(self) -> Mapping[int, str]:
        """This example is pared down to only 3 fovs, which are mapped to sequential integers"""
        return {
            0: "0",
        }
    def coordinate_map(self, round_: int, z: int):

        x_size, z_size, y_size = self.osmfish_metadata['ImageProperties']['PixelSize']
        x_pos, y_pos, z_pos = self.osmfish_metadata['TilesPositions']['Hybridization1'][0].values()
        return {
            Coordinates.X: (x_pos, x_pos + x_size),
            Coordinates.Y: (y_pos, y_pos + y_size),
            Coordinates.Z: (z_pos, z_pos + z_size),
        }

    def get_tile(
            self, fov_id: int, round_label: int, ch_label: int, zplane_label: int) -> FetchedTile:
        fov = self.fov_map[fov_id]
        target = 'dots'
        basename =  f"Hybridization{round_label + 1}_{target}_fov_{fov}.npy"
        file_path = os.path.join(self.input_dir, basename)
        coordinates = self.coordinate_map(round_label, zplane_label)
        return osmFISHTile(file_path, coordinates, zplane_label)

In [7]:
#@click.command()
#@click.argument("input-dir", type=str)
#@click.argument("metadata-yaml", type=str)
#@click.argument("output-dir", type=str)
def cli(input_dir, metadata_yaml, output_dir):
    """Reads osmFISH images from <input-dir> and experiment metadata from <metadata-yaml> and writes
    spaceTx-formatted data to <output-dir>.

    Raw data (input for this tool) for this experiment can be found at:
    s3://spacetx.starfish.data.upload/simone/

    Processed data (output of this tool) can be found at:
    s3://spacetx.starfish.data.public/20181031/osmFISH/
    """
    os.makedirs(output_dir, exist_ok=True)
    primary_tile_fetcher = osmFISHTileFetcher(os.path.expanduser(input_dir), metadata_yaml)
    nuclei_tile_fetcher = NucleiTileFetcher(os.path.expanduser(input_dir), metadata_yaml)
   # dots_tile_fetcher = DotsTileFetcher(os.path.expanduser(input_dir), metadata_yaml)

    # This is hardcoded for this example data set
    primary_image_dimensions = {
        Axes.ROUND: 3,
        Axes.CH:2,# len(primary_tile_fetcher.channel_map),
        Axes.ZPLANE: primary_tile_fetcher.num_z
    }
    
    aux_image_dimensions: Mapping[str, Mapping[Union[str, Axes], int]] = {
    "nuclei": {
        Axes.ROUND: 3,
        Axes.CH: 1,
        Axes.ZPLANE: 1,
    },
   # "dots": {
   #     Axes.ROUND: 1,
   #     Axes.CH: 1,
   #     Axes.ZPLANE:1,
   # },
}

    def postprocess_func(experiment_json_doc):
        experiment_json_doc["codebook"] = "codebook.json"
        return experiment_json_doc

    
    
    write_experiment_json(
        path=output_dir,
        fov_count=len(primary_tile_fetcher.fov_map),
        tile_format=ImageFormat.TIFF,
        primary_image_dimensions=primary_image_dimensions,
        aux_name_to_dimensions=aux_image_dimensions,
        primary_tile_fetcher=primary_tile_fetcher,
        aux_tile_fetcher={"nuclei": nuclei_tile_fetcher}, # "dots": dots_tile_fetcher},
        postprocess_func=postprocess_func,
        dimension_order=(Axes.ROUND, Axes.CH, Axes.ZPLANE)
    )
    with open(os.path.join(output_dir, "codebook.json"), "w") as f:
        codebook = primary_tile_fetcher.generate_codebook()
        json.dump(codebook, f)
    pass


if __name__ == "__main__":
    input_ = 'Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/prepared/'
    metadata = 'Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/prepared/Experimental_metadata.yml'
    output_ = 'Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/output/'
    cli(input_,metadata, output_)

C:\Users\tubau.n\.conda\envs\starfish\lib\site-packages\ipykernel_launcher.py:24: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
C:\Users\tubau.n\.conda\envs\starfish\lib\site-packages\ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """
